- MNIST Processing scripts from: https://gist.github.com/akesling/5358964

In [ ]:
# Imports
import os
import struct
import shutil
import random

import requests
import gzip
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

%matplotlib inline

In [ ]:

def mkdir(directory):
    if os.path.exists(directory):
        return
    os.mkdir(directory)

def download(url, destination):
    if os.path.exists(destination):
        print "File {} already exists. Skipping download".format(destination)
        return
    
    print "Downloading {} to {}".format(url, destination)
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(destination, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)   
        print "Download complete!"
    else:
        raise ValueError("Got a non-200 response")

def gz_decompress(source, dest):
    if os.path.exists(dest):
        print "{} already exists. Skipping decompressing".format(dest)
        return 
    print "Decompressing {} to {}".format(source, dest)
    with gzip.open(source, 'rb') as reader, open(dest, "wb") as writer:
        writer.write(reader.read())
    print "Done!"
        
def download_mnist(target_dir="./"):
    target_dir = os.path.join(target_dir, "mnist")
    mkdir(target_dir)
    
    download("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",  
             os.path.join(target_dir, "train-images-idx3-ubyte.gz"))
    gz_decompress(os.path.join(target_dir, "train-images-idx3-ubyte.gz"), 
                  os.path.join(target_dir, "train-images-idx3-ubyte"))
    
    download("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",  
             os.path.join(target_dir, "train-labels-idx1-ubyte.gz"))
    gz_decompress(os.path.join(target_dir, "train-labels-idx1-ubyte.gz"), 
                  os.path.join(target_dir, "train-labels-idx1-ubyte"))
    
    download("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz",  
             os.path.join(target_dir, "t10k-images-idx3-ubyte.gz"))
    gz_decompress(os.path.join(target_dir, "t10k-images-idx3-ubyte.gz"), 
                  os.path.join(target_dir, "t10k-images-idx3-ubyte"))
    
    download("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz",  
             os.path.join(target_dir, "t10k-labels-idx1-ubyte.gz"))
    gz_decompress(os.path.join(target_dir, "t10k-labels-idx1-ubyte.gz"), 
                  os.path.join(target_dir, "t10k-labels-idx1-ubyte"))

def read_mnist_full(dataset = "training", path = "./mnist", selected_labels=None):
    if dataset is "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError, "dataset must be 'testing' or 'training'"

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)
    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    return lbl, img
    
def read_mnist(dataset = "training", path = "./mnist", selected_labels=None):
    lbl, img = read_mnist_full(dataset, path, selected_labels)
    
    get_img = lambda idx: (lbl[idx], img[idx])

    # random permutation of indices
    p = np.random.permutation(len(lbl))
    lbl = lbl[p]
    img = img[p]
    
    # Create an iterator which returns each image in turn
    for i in xrange(len(lbl)):
        if selected_labels != None and lbl[i] not in selected_labels:
            continue
        yield get_img(i)
        

def show_mnist(image, title=None, fig_size=None):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """
    if not fig_size:
        fig_size = (5, 5)
    fig = plt.figure(figsize=fig_size)
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap=plt.cm.gray)
    imgplot.set_interpolation('nearest')
    ax.set_xticks([])
    ax.set_yticks([])
    #ax.xaxis.set_ticks_position('top')
    #ax.yaxis.set_ticks_position('left')
    if title:
        plt.title(title)
    plt.show()
    

# Introduction to Machine Learning
#### Samarth Bhargav
#### https://github.com/samarthbhargav/workshops


## Topics
- Machine Learning Concepts
- Supervised Learning
- Classification and Regression
- Visualizing Data with Matplotlib
- Logistic Regression


# What is Machine Learning?


### Siri / Google Now / Alexa / Cortana

#### Speech -> Text -> 'Understand' -> Response

### Optical Character Recognition


### Face recognition

# What *isn't* machine learning

# Classification

Examples!

# Regression 

Examples!

TODO List:
- ~~KNN~~
- Logistic Regression
- Decision Trees
- Iris, Titanic, MNIST

# The MNIST challenge!

##### Time to start coding!

In [ ]:
# download the MNIST data, decompress it
download_mnist()

# read a couple of sample images
sample = 100
sample_images = []
sample_labels = []
mnist_reader = read_mnist()
for i in range(sample):
    lbl, img = next(mnist_reader)
    sample_images.append(img)
    sample_labels.append(lbl)

In [ ]:
idx = random.randint(0, sample - 1)
show_mnist(sample_images[idx],sample_labels[idx])

In [ ]:
# each image is a 28x28 image
def plot_n(mult, mnist_reader):
    imgs = np.zeros(((28*mult), (28*mult)), dtype=np.uint8)

    for i in range(mult):
        for j in range(mult):
            lbl, img = next(mnist_reader)
            imgs[i*28:(i+1)*28, j*28:(j+1)*28] = img
    
    return imgs

In [ ]:
imgs = plot_n(20, read_mnist())
show_mnist(imgs)

In [ ]:
# variance in the labels makes the problem non-trivial
selected_labels = set([2])
imgs = plot_n(20, read_mnist(selected_labels=selected_labels))
show_mnist(imgs)

In [ ]:
selected_labels = set([8, 9])
imgs = plot_n(20, read_mnist(selected_labels=selected_labels))
show_mnist(imgs)

## How would you solve it?

How does the computer see it?


In [ ]:
# what does the computer 'see'?
def ascii_show(image):
    for y in image:
        row = ""
        for x in y:
            row += '{0: <4}'.format(x)
        print row

In [ ]:
idx = random.randint(0, sample - 1)
ascii_show(sample_images[idx])

# Algorithm : K-Nearest Neighbours

Given a sample to classify, we get the K-nearest neighbours. The most represented class in the K-nearest neighbours is the prediction

# Aside: How can we compute distance between two vectors?

In [ ]:
def plot_points(points, labels=None, xlim=(0, 10), ylim=(0, 10)):
    points = np.array(points)
    # since it's two dimensions, they can be plotted!
    plt.plot(points[:, 0], points[:, 1], "bo")
    plt.xlim(xlim[0], xlim[1])
    plt.ylim(ylim[0], ylim[1])
    if labels:
        for p, l in zip(points, labels):
            plt.text(p[0] + .2, p[1] + 0.1, "${}$".format(l))

In [ ]:
# let's take 3 arrays for example
a = np.array([2, 3])
b = np.array([3, 7])
c = np.array([1, 1])

plot_points([a, b, c], ["a", "b", "c"])

In [ ]:
# distance metric 1
# taxi-cab / manhattan distance
Image(url="http://www.joachimdespland.com/img/mammoth/pathfinding.fig2.png")

In [ ]:
np.abs(a - b).sum()

In [ ]:
def manhattan(x, y): 
    # this also works for >2 dimensions
    return np.abs(x - y).sum()

In [ ]:
# it's symmetric, so we can compute only once
manhattan(a, b), manhattan(b, a)

In [ ]:
print "Distance between a and b:", manhattan(a,b)
print "Distance between a and c:", manhattan(a,c)
print "Distance between b and c:", manhattan(b,c)

In [ ]:
# now let's take a look at the euclidean distance
Image("http://rosalind.info/media/Euclidean_distance.png")

In [ ]:
# it's the 'straight line ' distance between 2 points
np.sqrt(((a-b)**2).sum())

In [ ]:
def euclidean(x, y):
    return np.sqrt(((x-y)**2).sum())

In [ ]:
# it's symmetric, so we can compute only once
euclidean(a, b), euclidean(b, a)

In [ ]:
print "Distance between a and b:", euclidean(a,b)
print "Distance between a and c:", euclidean(a,c)
print "Distance between b and c:", euclidean(b,c)

## Back to K-NN!

In [ ]:
def find_nearest(points, cmp_point, point_index, k, distance_function):
    distances = []
    for idx, point in enumerate(points):
        if idx == point_idx:
            continue
        distances.append((idx, distance_function(cmp_point, points[idx])))
    distances.sort(key=lambda d: d[1])
    return distances[:k]

a = np.array([2, 3])
b = np.array([3, 7])
c = np.array([1, 1])

points = [a, b, c]
point_idx = 2

idx, distance = find_nearest(points, points[point_idx], point_idx, 1, euclidean)[0]
print "Point nearest to {} is {}".format(points[point_idx], points[idx])

In [ ]:
# now, let's try out K-NN for the MNIST dataset
labels, images = read_mnist_full()

In [ ]:
# shape tells us the dimensions of the data
labels.shape

In [ ]:
images.shape

In [ ]:
# since it's a 28x28 matrix, the images have to be reshaped to make it work 
images = images.reshape(60000, 28*28)

In [ ]:
# we can get back the original data by reshaping it again to 28x28
show_mnist(images[0].reshape(28, 28))

In [ ]:
# now, we can use the find_nearest neighbours
# pick an image at random
index = np.random.randint(0, len(images))
img, lbl = images[index], labels[index]

show_mnist(img.reshape(28, 28))

nearest_neighbours = find_nearest(images, images[index], index, 10, manhattan)

In [ ]:
# from this list, extract corresponding votes and tally it up!
nn_labels = [labels[i] for i, d in nearest_neighbours]
nn_labels

In [ ]:
# there's a handy class called Counter we can use
from collections import Counter
counts = Counter(nn_labels)
counts

In [ ]:
counts.most_common(3)

In [ ]:
prediction = counts.most_common(1)[0][0]
prediction

In [ ]:
# let's pull all of this into a nice function
def predict(index, images, k, distance_function):
    nearest_neighbours = find_nearest(images, images[index], index, k, distance_function)
    nn_labels = [labels[i] for i, d in nearest_neighbours]
    counts = Counter(nn_labels)
    return counts.most_common(1)[0][0]

# this is for prediction if we have a new image
def predict_image(img, images, labels, k, distance_function):
    nearest_neighbours = find_nearest(images, img, -1, k, distance_function)
    nn_labels = [labels[i] for i, d in nearest_neighbours]
    counts = Counter(nn_labels)
    return counts.most_common(1)[0][0]

In [ ]:
index = np.random.randint(0, len(images))
img, lbl = images[index], labels[index]

show_mnist(img.reshape(28, 28))
predict(index, images, 10, manhattan)

# Evaluation of Classification Models

In [ ]:
# assumption: let's assume there are only two classes - 0 and 1 
# this is called a binary classification problem, btw

# let's take these as examples
y_actual =    [0,1,0,0,0,0,1,1,0,1]
y_predicted = [1,0,0,0,0,1,1,1,0,1]

# is this good / bad?

In [ ]:
# simplest metric ever!
def accuracy(actual, predicted):
    assert len(predicted)  == len(actual)
    correct = 0.0
    for a, p in zip(actual, predicted):
        if a == p:
            correct += 1
    return correct / len(actual)
accuracy(y_actual, y_predicted)

In [ ]:
# problem with accuracy :(
# let's say we have an unbalanced dataset in which almost all points are 1 
unbalanced_y_actual = [1,1,1,1,1,1,1,1,0,1]
# let's say we have a classifier which is a single line of code that predicts '1' all the time!
unbalanced_y_predicted = [1,1,1,1,1,1,1,1,1,1]

accuracy(unbalanced_y_actual, unbalanced_y_predicted)

#### To tackle this, we can come up with more metrics! 
Let's first create a helpful matrix of all possible results in a binary classification setting:

||Labels| $0$  | $1$  | $Predicted$  |
|---|---|---|---|
|$Actual$| $0$     | $True Negative$  | $False Positive$  | 
|$Actual$| $1$     |  $False Negative$ |  $True Positive$ |  


We can now define precision and recall:
In a binary setting, 

Precision is the fraction of correctly predicted values - $1$s among all the values we've **predicted** as $1$. 

Recall is the fraction of correctly predicted values - $1$s over **total number** of $1$s instances in the image.


Can you guess:

$Precision = ?$

$Recall = ?$

$Precision = \frac{TP}{(TP  + FP)}$

$Recall = \frac{TP}{(TP  + FN)}$

### Precision and recall intuition
A system with high recall but low precision returns many results, but most of its predicted labels are incorrect when compared to the training labels.

A system with high precision but low recall is just the opposite, returning very few results, but most of its predicted labels are correct when compared to the training labels.

An ideal system with high precision and high recall will return many results, with all results labeled correctly.

In [ ]:
# count true positives 
def true_positive(actual, predicted, pos_label):
    tp = 0
    for a, p in zip(actual, predicted):
        if a == pos_label and a == p:
            tp += 1
    return tp

def true_negative(actual, predicted, pos_label):
    tn = 0
    for a, p in zip(actual, predicted):
        if a != pos_label and a == p:
            tn += 1
    return tn

def false_positive(actual, predicted, pos_label):
    fp = 0
    for a, p in zip(actual, predicted):
        if a != pos_label and p == pos_label:
            fp += 1
    return fp

def false_negative(actual, predicted, pos_label):
    fn = 0
    for a, p in zip(actual, predicted):
        if a == pos_label and p != pos_label:
            fn += 1
    return fn

def confusion_matrix(actual, predicted, pos_label):
    tp = true_positive(actual, predicted, pos_label)
    tn = true_negative(actual, predicted, pos_label)
    fp = false_positive(actual, predicted, pos_label)
    fn = false_negative(actual, predicted, pos_label)
    return [
        [tn, fp],
        [fn, tp]
    ]

def precision(actual, predicted, pos_label):
    tp = true_positive(actual, predicted, pos_label)
    fp = false_positive(actual, predicted, pos_label)
    if tp == 0:
        return 0
    return float(tp) / (tp + fp)
    
    
def recall(actual, predicted, pos_label):
    # fill this up!
    return 0

print precision(y_actual, y_predicted, 1)
print recall(y_actual, y_predicted, 1)

In [ ]:
print precision(unbalanced_y_actual, unbalanced_y_predicted, 1)
print recall(unbalanced_y_actual, unbalanced_y_predicted, 1)

In [ ]:
print precision(unbalanced_y_actual, unbalanced_y_predicted, 0)
print recall(unbalanced_y_actual, unbalanced_y_predicted, 0)

In [ ]:
# play around with Precision and Recall
actual = [1, 1, 0, 0]
predicted = [1, 1, 0, 1]
print precision(actual, predicted, 1)
print recall(actual, predicted, 1)

# One metric to rule them all!

$F1$ score is the harmonic mean of Precision & Recall

$F1 = 2\frac{P . R}{P+R} $

In [ ]:
def f1_score(actual, predicted, pos_label):
    # fill this up!
    return 0

print "Balanced: {}".format(f1_score(actual, predicted, 1))
print "Unbalanced: {}".format(f1_score(actual, predicted, 1))

In [ ]:
def print_all_metrics(y_actual, y_predicted):
    for label in np.unique(y_actual):
        print "LABEL: {}".format(label)
        print "\tAccuracy: ", accuracy(y_actual, y_predicted)
        print "\tTrue Positive: ", true_positive(y_actual, y_predicted, label)
        print "\tTrue Negative: ", true_negative(y_actual, y_predicted, label)
        print "\tFalse Positive: ", false_positive(y_actual, y_predicted, label)
        print "\tFalse Negative: ", false_negative(y_actual, y_predicted, label)
        print "\tConfusion Matrix: ", confusion_matrix(y_actual, y_predicted, label)
        print "\tPrecision: ", precision(y_actual, y_predicted, label)
        print "\tRecall: ", recall(y_actual, y_predicted, label)
        print "\tF1 Score: ", f1_score(y_actual, y_predicted, label)
        print
print_all_metrics(y_actual, y_predicted)

In [ ]:
# now let's take a look at the unbalanced example
print_all_metrics(unbalanced_y_actual, unbalanced_y_predicted)

In [ ]:
# let's select a sample, because prediction on the whole data takes a loooong time!
idx_ = np.random.permutation(range(0, len(images)))[:100]
predicted_eval_sample = []
for i in idx_:
    predicted_eval_sample.append(predict(i, images, 10, euclidean))

In [ ]:
print_all_metrics(labels[idx_], predicted_eval_sample)

# Precision vs Recall vs F1 Score 

Which is more imporant?

- Spam
- Biometric scanning
- Credit card fraud


# Generalization

### Problem 1. How can you confirm whether your model works in practice? 
### Problem 2. How can I select *hyperparameters* for my model?

In [ ]:
Image("./sample_space.png")

## How can you confirm whether your model works in practice?
### Solution: Separate your data into 2 sets - the train and the test set

Call one a training set and use it for training your data. This is typically 60-80% of your data (depending on the # samples you have)
The other set - the test set - is only touched *once* when you evaluate your *final* model. 


As long as the # samples in the test set is large enough (and r, we can safely assume that efficacy of the model in the test set is approximately equal to the efficacy of the model on the 'real world' data.

## Problem 2. How can I select *hyperparameters* for my model?
### Solution for #2: Separate your *train* data into 2 sets: the train and validation set. The test set remains untouched

The first set, as the name suggests, will be used for training. This is typically 50-60% of the total data.
The second set, will be used to tune hyperparameters. This is typically 10-20% of the total data.

So in essense, there are 3 sets of data:
- Train Set - 50-60% of the total data
- Validaton Set - 10-20% of the total data
- Test Set - 20-40% of the total data


# Hyperparameter tuning

- Select a metric most important for you (ex precision)


- For each set of parameters, fit your model
- Evalute model using validation set
- Store evaluation


- Pick the best performing model on the validation set as your final model

In [ ]:
# split data 
total_size = len(images)

train_size = int(0.6 * total_size)
validation_size = 100 #int(0.2 * total_size)
print train_size, validation_size, (total_size-train_size-validation_size)
shuffle_idx = np.random.permutation(range(0, len(images)))

In [ ]:
x_train, y_train = images[shuffle_idx[0: train_size]], labels[shuffle_idx[0: train_size]]
print x_train.shape, y_train.shape

In [ ]:
x_valid = images[shuffle_idx[train_size: train_size + validation_size]]
y_valid = labels[shuffle_idx[train_size: train_size + validation_size]]
print x_valid.shape, y_valid.shape

In [ ]:
x_test = images[shuffle_idx[train_size + validation_size: ]]
y_test = labels[shuffle_idx[train_size + validation_size: ]]
print x_valid.shape, y_valid.shape

In [ ]:
# let's pick precision as the metric
selection_metric = precision


parameter_grid = [
    {
        "k": 1,
        "distance": euclidean
    },
    {
        "k": 5,
        "distance": euclidean
    },
    {
        "k": 10,
        "distance": euclidean
    },
    {
        "k": 1,
        "distance": manhattan
    },
    {
        "k": 5,
        "distance": manhattan
    },
    {
        "k": 10,
        "distance": manhattan
    }
]


evaluated = []
for param_dict in parameter_grid:
    print "Trying Parameter: {}".format(param_dict)
    predictions = []
    # compute predictions for the validation set
    for i, img in enumerate(x_valid):
        # print a progress report once in a while
        if i % 10 == :
            print "\t{} of {} images".format(i, len(x_valid))
        # use only the train images and labels here! 
        predictions.append(predict_image(img, x_train, y_train, param_dict["k"], param_dict["distance"]))
    
    per_label_selection_metrics = []
    for label in np.unique(labels[idx_]):
        per_label_selection_metrics.append(selection_metric(y_valid, predictions, label))
    evaluated.append({
            "params" : param_dict,
            "metric": np.mean(per_label_selection_metrics)
        })
    
    
for p in evaluated:
    print p

In [ ]:
best_params = max(evaluated, key=lambda _: _["metric"])
best_params

In [ ]:
best_k = best_params["params"]["k"]
best_dist = best_params["params"]["distance"]

## Just a reminder: *Never* touch the test set except only once in your entire process 

In [ ]:
t_idx = 100
y_test_pred = [predict_image(img, x_train, y_train, best_k, best_dist) for img in x_test[:t_idx]]

test_eval = []
for label in np.unique(labels):
    test_eval.append(selection_metric(y_test[:t_idx], y_test_pred, label))
    
print np.mean(test_eval)

## Problems with KNN

Pros?
Cons?

# In Summary

**Data preparation and exploration** (more on this later)
- Explore the data. Note any biases and properties of the data that may affect modeling and evaluation (skewed class distribution, high or low variance, etc).
- Decide on a metric
- Decide which algorithm(s) to use (more on this later)
- Prepare dataset: Convert it into X & y matrices

**Modeling Phase**
- Split the dataset into a training, validation and test set (more on this later)
- Train the models(s) on the training set and tune hyperparameters
- Repeat until you get a good score on the validation set.

**Test step**
- Compute test accuracy. Remember this has to be done only once. 

# Scikit-Learn

Everything we just did can be reduced to a few lines of code if we use `scikit-learn`.

In [ ]:
from sklearn.metrics import precision_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric
# data preparation
labels, images = read_mnist_full()
images = images.reshape(60000, 28*28)
print labels.shape, images.shape

# create train test split

# 20% test set
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

# get only the first 5000 images
x_train, y_train = x_train[:5000], y_train[:5000]

# get only the first 1000 images 
x_test, y_test = x_test[:1000], y_test[:1000]

print "Train:", x_train.shape, y_train.shape
print "Test:", x_test.shape, y_test.shape

parameter_grid = {
    "metric": [DistanceMetric.get_metric("euclidean"), 
               DistanceMetric.get_metric("manhattan")],
    "n_neighbors": [1, 5, 10]
}

clf = GridSearchCV(KNeighborsClassifier(algorithm = 'ball_tree'), parameter_grid, 
             scoring=make_scorer(precision_score, average="macro"),n_jobs=3, cv=2, verbose=2)
clf.fit(x_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
precision_score(y_test, clf.predict(x_test), average="macro")